# Lab 6 

In [1]:
import spacy
nlp = spacy.load("en_core_web_md")

2022-11-18 00:04:04.265271: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import requests
from datetime import datetime

In [3]:
api_key = "764915bdfe257c96cfbc0cf145272cbd"

city = "Evanston"
api_url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
r = requests.get(api_url).json()
print(r)
print(r.keys())

{'coord': {'lon': -87.6901, 'lat': 42.0411}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 271.55, 'feels_like': 265.85, 'temp_min': 270.51, 'temp_max': 272.25, 'pressure': 1019, 'humidity': 69}, 'visibility': 10000, 'wind': {'speed': 5.66, 'deg': 260}, 'clouds': {'all': 75}, 'dt': 1668751449, 'sys': {'type': 2, 'id': 2038650, 'country': 'US', 'sunrise': 1668775477, 'sunset': 1668810455}, 'timezone': -21600, 'id': 4891382, 'name': 'Evanston', 'cod': 200}
dict_keys(['coord', 'weather', 'base', 'main', 'visibility', 'wind', 'clouds', 'dt', 'sys', 'timezone', 'id', 'name', 'cod'])


In [4]:
def K2F(temp_K):
    temp_F = round((temp_K - 273.15) * 9/5 + 32,0)
    return temp_F

def K2C(temp_K):
    temp_C = temp_K - 273.15
    return temp_C

In [5]:
print('Weather description: ',r["weather"][0]["description"])
print('Temperature (F): ',K2F(r["main"]["temp"]))
print('Temperature (C): ',K2C(r["main"]["temp"]))

Weather description:  broken clouds
Temperature (F):  29.0
Temperature (C):  -1.599999999999966


In [6]:
def get_data(city_name):
    api_url = f"http://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={api_key}"

    response = requests.get(api_url)
    response_dict = response.json()
    return response_dict

In [7]:
def get_weather(city_name):
    response_dict = get_data(city_name)

    weather, temp = response_dict["weather"][0]["description"], str(K2F(response_dict["main"]["temp"])) 
    
    return weather, temp

In [8]:
def get_sunrise(city_name):
    response_dict = get_data(city_name)
    sunrise_ts = response_dict["sys"]["sunrise"]
    dt_o = datetime.fromtimestamp(1668689003)
    return "%d:%d" % (dt_o.hour, dt_o.minute)

In [9]:
weather, temp = get_weather("London")
print("Weather: ",weather, "\nTemperature: ",temp)

Weather:  overcast clouds 
Temperature:  47.0


In [10]:
sunrise = get_sunrise("Evanston")
print(sunrise)

6:43


In [11]:
weather_request = "What is the weather?"
weather_req_doc = nlp(weather_request)

In [12]:
statement_1 = "How cold is it in London?"
statement_2 = "Select flights to London"
statement1_doc = nlp(statement_1)
statement2_doc = nlp(statement_2)
print(weather_req_doc.similarity(statement1_doc))
print(weather_req_doc.similarity(statement2_doc))

0.7867970023620359
0.28759060509276346


In [13]:
for ent in statement1_doc.ents:
    print("{} -> {}".format(ent.text,ent.label_))

London -> GPE


In [14]:
def extract_city(statement):
    city = None
    for ent in statement.ents:
        if ent.label_ == "GPE": # GeoPolitical Entity
            city = ent.text
            break
    return city

In [15]:
def responder_weather(statement):
    statement = nlp(statement)
    city = extract_city(statement)
    if city == None:
        return "You need to tell me a city to check."

    city_weather, city_temp = get_weather(city)
    if city_weather is not None:
        return "In " + city + ", the current weather is: " + city_weather + \
        ", and the air temperature is: " + city_temp + " Fahrenheit"
    else:
        return "Something went wrong."

In [16]:
def responder_sunrise(statement):
    statement = nlp(statement)
    city = extract_city(statement)
    if city == None:
        return "You need to tell me a city to check."
    sunrise = get_sunrise(city)
    return "Sun rises at %s" % sunrise

In [17]:
class ChatBot:
    def __init__(self, template_responder_pairs):
        self.template_responder_pairs = template_responder_pairs
        self.template_nlp_responder_pairs = \
            [(nlp(template_responder_pair[0]), template_responder_pair[1]) 
             for template_responder_pair in template_responder_pairs]
    
    def respond(self, inp):
        inp_nlp = nlp(inp)
        similarity_responder_pairs = \
            [(template_nlp_responder_pair[0].similarity(inp_nlp), template_nlp_responder_pair[1]) 
             for template_nlp_responder_pair in self.template_nlp_responder_pairs]
        similarity_responder_pairs = sorted(similarity_responder_pairs, key=lambda p: p[0], reverse=True)
        if similarity_responder_pairs[0][0] < 0.5:
            return "Sorry I don't understand that. Try one of the following:\n" + \
                "\n".join([template_responder_pair[0] 
                           for template_responder_pair in self.template_responder_pairs])
        
        return similarity_responder_pairs[0][1](inp)

In [18]:
chatbot = ChatBot([("What is the weather?", responder_weather), ("When does the sun rise?", responder_sunrise)])

In [19]:
response_1 = chatbot.respond("Is it freezing in Amsterdam?")
print(response_1)

In Amsterdam, the current weather is: clear sky, and the air temperature is: 46.0 Fahrenheit


In [20]:
response_2 = chatbot.respond("How cold is it?")
print(response_2)

You need to tell me a city to check.


In [21]:
response_3 = chatbot.respond("When is the sunrise in Evanston?")
print(response_3)

In Evanston, the current weather is: broken clouds, and the air temperature is: 29.0 Fahrenheit


In [22]:
response_4 = chatbot.respond("When does the sun rise in San Diego?")
print(response_4)

Sun rises at 6:43
